In [44]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import LabelEncoder
from keras.layers import LSTM,Dense,Dropout, Reshape, Conv1D, GlobalMaxPooling1D, MaxPooling1D, Flatten
from keras.models import Sequential
from keras.layers import Embedding,SimpleRNN
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils

from tensorflow.keras.optimizers import Adam

from gensim.models import Word2Vec

In [45]:
from google.cloud import storage
import os
import io
client = storage.Client()
bucket = client.get_bucket('nlp_final_data')

#blob = bucket.blob('data_no_stop.csv')
blob = bucket.blob('songs_nGrams.csv')
content = blob.download_as_string()

df_1grams = pd.read_csv(io.BytesIO(content))

KeyboardInterrupt: 

In [52]:
df['data_tokenized'] = df['data_tokenized'].apply(lambda x: x.strip('[]').replace('\'', '').split(', '))

In [54]:
df['num_tokens'] = df['1-grams'].apply(lambda x: len(x))

In [55]:
df_new = df[df.num_tokens<1000]

In [56]:
df_new = df_new[df_new.num_tokens>10]

In [61]:
df_new.sort_values(by=['num_tokens'],ascending=False)

,Unnamed: 0,title,tag,artist,tokens_no_stop,1-grams,num_tokens
42727,42727,Math Hoffa vs. John John Da Don,rap,URLtv,"['make', 'worst', 'life', 'decisions', 'ive', ...","[make, worst, life, long, virus, battle, rap, ...",978
8706,8706,Ridin Dirty VS. The Chronic,rap,Rap Genius Users,"['truthisgravey', 'ridin', 'dirty', 'ugk', 'op...","[dirty, opening, statement, underground, bette...",964
79287,79287,Invitation,rap,Aha Gazelle,"['youre', 'real', 'youre', 'playing', 'dont', ...","[real, dont, believe, word, saying, think, foo...",960
20382,20382,Block Cypher,rap,EC1 (Bourne Estate),"['put', 'holes', 'tops', 'gang', 'got', 'opps'...","[put, top, gang, got, co, step, waist, get, fi...",958
71344,71344,100 Bulletz vs JC,rap,King of the Dot,"['bulletz', 'vs', 'julian', 'carter', 'lets', ...","[carter, set, every, battle, gon, knock, boy, ...",951
...,...,...,...,...,...,...,...
190621,190621,Ten Tables,rock,Goon,"['way', 'home', 'mottled', 'vine', 'pain', 'pa...","[way, home, mottled, vine, pain, pin, nine, il...",11
359735,359735,Man The Harpoons,country,The Lost Episode,"['instrumental', 'track', 'whilst', 'supposed'...","[instrumental, track, whilst, supposed, decide...",11
179290,179290,I Told You,rock,Frst,"['mist', 'closing', 'mist', 'closing', 'like',...","[mist, mist, like, ghost, cheek, like, like, o...",11
228388,228388,Nerve Gas,rock,Guided by Voices,"['must', 'master', 'nerve', 'shall', 'must', '...","[must, master, nerve, shall, must, slit, dead,...",11


In [71]:
X = df['data_tokenized']
y = df['tag']

In [72]:
# split the balanced data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2,stratify=y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=42)

In [74]:
tokenize = Tokenizer(char_level=False)
tokenize.fit_on_texts(X) # only fit on train

# we will need this later
num_words = len(tokenize.word_index)+1

In [75]:
num_words

45920

In [76]:
x_train = tokenize.texts_to_sequences(X_train)
x_test = tokenize.texts_to_sequences(X_test)

encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

num_classes = np.max(y_train) + 1
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [77]:
max_len = 0
for song in x_train:
    if len(song)>max_len:
        max_len = len(song)

In [78]:
max_len

978

In [79]:
max_log_length = max_len
x_train = pad_sequences(x_train, maxlen=max_log_length)
x_test = pad_sequences(x_test, maxlen=max_log_length)

In [80]:
model3 = Sequential()

model3.add(Embedding(input_dim=num_words,output_dim=64,input_length=max_log_length))

num_filters = 128
kernel_sizes = [2,3, 4]
for kernel_size in kernel_sizes:
    model3.add(Conv1D(num_filters, kernel_size, activation='relu'))
#model3.add(Conv1D(128, 3, activation='relu'))

model3.add(MaxPooling1D(pool_size=2))
model3.add(LSTM(units=64,recurrent_dropout=0.5))
model3.add(Dropout(0.5))

# Add Dense layers
hidden_units = [128, 64]
for units in hidden_units:
    model3.add(Dense(units, activation='relu'))
    model3.add(Dropout(0.5))
    
model3.add(Dense(num_classes,activation='softmax'))

# compile the model
model3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [81]:
history = model3.fit(x_train, y_train,
                    batch_size=128,
                    epochs=3,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/3
2421/2421 [==============================] - 1807s 745ms/step - loss: 1.2583 - accuracy: 0.4671 - val_loss: 1.1062 - val_accuracy: 0.5559
Epoch 2/3
 817/2421 [=========>....................] - ETA: 19:17 - loss: 1.1129 - accuracy: 0.5588

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



2421/2421 [==============================] - 1797s 742ms/step - loss: 1.0382 - accuracy: 0.5892 - val_loss: 1.0247 - val_accuracy: 0.5865


In [82]:
model3.save('model_cleaned_data.h5')

In [22]:
blob_name = 'model_final_05_20.h5'
client = storage.Client()

blob = bucket.blob(blob_name)
blob.upload_from_filename(blob_name)

In [29]:
model3 = Sequential()

model3.add(Embedding(input_dim=num_words,output_dim=128,input_length=max_log_length))

num_filters = 128
kernel_sizes = [2,3]
for kernel_size in kernel_sizes:
    model3.add(Conv1D(num_filters, kernel_size, activation='relu'))
#model3.add(Conv1D(128, 3, activation='relu'))

model3.add(MaxPooling1D(pool_size=5))
model3.add(LSTM(units=64,recurrent_dropout=0.5))
model3.add(Dropout(0.5))

# Add Dense layers
hidden_units = [128, 64]
for units in hidden_units:
    model3.add(Dense(units, activation='relu'))
    model3.add(Dropout(0.5))
    
model3.add(Dense(num_classes,activation='softmax'))

# compile the model
model3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [30]:
history = model3.fit(x_train, y_train,
                    batch_size=128,
                    epochs=3,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/3
2438/2438 [==============================] - 986s 403ms/step - loss: 1.1825 - accuracy: 0.5121 - val_loss: 1.0520 - val_accuracy: 0.5676
Epoch 2/3
2438/2438 [==============================] - 983s 403ms/step - loss: 1.0486 - accuracy: 0.5841 - val_loss: 1.0328 - val_accuracy: 0.5752
Epoch 3/3
2438/2438 [==============================] - 984s 404ms/step - loss: 0.9983 - accuracy: 0.6038 - val_loss: 1.0325 - val_accuracy: 0.5831


In [31]:
model3.save('model_best_05_20.h5')

In [32]:
blob_name = 'model_best_05_20.h5'
client = storage.Client()

blob = bucket.blob(blob_name)
blob.upload_from_filename(blob_name)

In [58]:
model3 = Sequential()

model3.add(Embedding(input_dim=num_words,output_dim=256,input_length=max_log_length))

num_filters = 128
kernel_sizes = [3,4,5]
for kernel_size in kernel_sizes:
    model3.add(Conv1D(num_filters, kernel_size, activation='relu'))
#model3.add(Conv1D(128, 3, activation='relu'))

model3.add(MaxPooling1D(pool_size=5))
model3.add(LSTM(units=128,recurrent_dropout=0.5))
model3.add(Dropout(0.5))

# Add Dense layers
hidden_units = [128, 64]
for units in hidden_units:
    model3.add(Dense(units, activation='relu'))
    model3.add(Dropout(0.5))
    
model3.add(Dense(num_classes,activation='softmax'))

# compile the model
model3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [59]:
history = model3.fit(x_train, y_train,
                    batch_size=128,
                    epochs=3,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/3
2438/2438 [==============================] - 1975s 806ms/step - loss: 1.1744 - accuracy: 0.5203 - val_loss: 1.0538 - val_accuracy: 0.5696
Epoch 2/3
2438/2438 [==============================] - 1961s 804ms/step - loss: 1.0488 - accuracy: 0.5834 - val_loss: 1.0276 - val_accuracy: 0.5815
Epoch 3/3
2438/2438 [==============================] - 1967s 807ms/step - loss: 1.0015 - accuracy: 0.6016 - val_loss: 1.0305 - val_accuracy: 0.5808


In [31]:
model3.save('model_best_05_20.h5')

In [32]:
blob_name = 'model_best_05_20.h5'
client = storage.Client()

blob = bucket.blob(blob_name)
blob.upload_from_filename(blob_name)

In [56]:
model3 = Sequential()

model3.add(Embedding(input_dim=num_words,output_dim=256,input_length=max_log_length))

num_filters = 128

model3.add(Conv1D(num_filters, 5, activation='relu'))
model3.add(MaxPooling1D(pool_size=5))

# model3.add(LSTM(units=128,recurrent_dropout=0.5))
# model3.add(Dropout(0.5))

model3.add(Conv1D(num_filters, 5, activation='relu'))
model3.add(MaxPooling1D(pool_size=5))

model3.add(Flatten())

# Add Dense layers
hidden_units = [128, 64]
for units in hidden_units:
    model3.add(Dense(units, activation='relu'))
    model3.add(Dropout(0.5))
    
model3.add(Dense(num_classes,activation='softmax'))

# compile the model
model3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [57]:
history = model3.fit(x_train, y_train,
                    batch_size=128,
                    epochs=3,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/3
2438/2438 [==============================] - 1242s 509ms/step - loss: 1.1571 - accuracy: 0.5260 - val_loss: 1.0405 - val_accuracy: 0.5794
Epoch 2/3
2438/2438 [==============================] - 1172s 481ms/step - loss: 0.9481 - accuracy: 0.6221 - val_loss: 1.0542 - val_accuracy: 0.5761


In [31]:
model3.save('model_best_05_20.h5')

In [32]:
blob_name = 'model_best_05_20.h5'
client = storage.Client()

blob = bucket.blob(blob_name)
blob.upload_from_filename(blob_name)

Using convolutional layers in a neural network for text data can be beneficial for several reasons:

Local Pattern Extraction: Convolutional layers can effectively capture local patterns and features in the text data. By applying filters of different sizes, the convolutional operation can detect patterns at various levels of granularity. This allows the model to learn relevant features such as n-grams, word combinations, or other local patterns that are indicative of the text's meaning or sentiment.

Translation Invariance: Convolutional layers exhibit translation invariance, which means they can recognize patterns regardless of their exact position in the input. In the context of text data, this property is useful because the position of a particular word or phrase in a sentence may not always be critical for understanding its meaning. By capturing patterns irrespective of their location, convolutional layers can provide robust representations that are not overly sensitive to word order.

Reduced Parameter Count: Convolutional layers can help reduce the number of parameters in the model compared to fully connected layers. This reduction is achieved by weight sharing through the use of filters. By sharing weights, the model can capture the same pattern or feature across different positions in the input, resulting in fewer trainable parameters. This parameter efficiency can make the model easier to train and less prone to overfitting, especially when dealing with limited amounts of text data.

Hierarchical Feature Learning: Deep architectures with multiple convolutional layers can learn hierarchical representations of the input text. Lower-level convolutional layers can capture basic local features, while higher-level convolutional layers can learn more complex combinations of these features. This hierarchical learning enables the model to capture both low-level and high-level semantic information from the text.

It's worth noting that while convolutional neural networks (CNNs) have primarily been associated with image processing tasks, they have been successfully adapted for natural language processing (NLP) tasks, including text classification. The convolutional operations in text CNNs are typically performed along the time dimension (i.e., word or character sequences) rather than across spatial dimensions (as in image CNNs).

That being said, the effectiveness of using convolutional layers in text classification tasks may vary depending on the specific dataset and problem. It's recommended to experiment with different architectures and compare the performance with other approaches, such as recurrent neural networks (RNNs) or transformers, to determine the best choice for your particular task.






Regenerate response